<a href="https://colab.research.google.com/github/MichaelOkoro24/Amazon_Vine_Analysis/blob/main/Amazon_Reviews_ETL.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import os
# Find the latest version of spark 3.0  from http://www.apache.org/dist/spark/ and enter as the spark version
# For example:
# spark_version = 'spark-3.3.0'
spark_version = 'spark-3.3.0'
os.environ['SPARK_VERSION']=spark_version

# Install Spark and Java
!apt-get update
!apt-get install openjdk-11-jdk-headless -qq > /dev/null
!wget -q http://www.apache.org/dist/spark/$SPARK_VERSION/$SPARK_VERSION-bin-hadoop3.tgz  
!tar xf $SPARK_VERSION-bin-hadoop3.tgz 
!pip install -q findspark

# Set Environment Variables
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-11-openjdk-amd64"
os.environ["SPARK_HOME"] = f"/content/{spark_version}-bin-hadoop3"

# Start a SparkSession
import findspark
findspark.init()

Hit:1 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease
Get:2 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Hit:3 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease
Hit:4 http://archive.ubuntu.com/ubuntu bionic InRelease
Get:5 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Hit:6 http://ppa.launchpad.net/cran/libgit2/ubuntu bionic InRelease
Ign:7 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Get:8 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease [1,581 B]
Hit:9 http://ppa.launchpad.net/deadsnakes/ppa/ubuntu bionic InRelease
Get:10 http://archive.ubuntu.com/ubuntu bionic-backports InRelease [83.3 kB]
Hit:11 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Get:12 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Packages [946 kB]
Hit:13 http://ppa

In [ ]:
# Download the Postgres driver that will allow Spark to interact with Postgres.
!wget https://jdbc.postgresql.org/download/postgresql-42.2.16.jar

--2022-10-03 21:35:19--  https://jdbc.postgresql.org/download/postgresql-42.2.16.jar
Resolving jdbc.postgresql.org (jdbc.postgresql.org)... 72.32.157.228, 2001:4800:3e1:1::228
Connecting to jdbc.postgresql.org (jdbc.postgresql.org)|72.32.157.228|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1002883 (979K) [application/java-archive]
Saving to: ‘postgresql-42.2.16.jar.1’

postgresql-42.2.16. 100%[===================>] 979.38K  1.13MB/s    in 0.8s    

2022-10-03 21:35:21 (1.13 MB/s) - ‘postgresql-42.2.16.jar.1’ saved [1002883/1002883]



In [ ]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("M16-Amazon-Challenge").config("spark.driver.extraClassPath","/content/postgresql-42.2.16.jar").getOrCreate()

In [ ]:
# Read in data from S3 Buckets 
from pyspark import SparkFiles
url = "https://s3.amazonaws.com/amazon-reviews-pds/tsv/amazon_reviews_us_Camera_v1_00.tsv.gz"
spark.sparkContext.addFile(url)
df = spark.read.option("encoding", "UTF-8").csv(SparkFiles.get(""), sep="\t", header=True, inferSchema=True)
df.show()

+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-------------------+
|marketplace|customer_id|     review_id|product_id|product_parent|       product_title|product_category|star_rating|helpful_votes|total_votes|vine|verified_purchase|     review_headline|         review_body|        review_date|
+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-------------------+
|         US|    2975964|R1NBG94582SJE2|B00I01JQJM|     860486164|GoPro Rechargeabl...|          Camera|          5|            0|          0|   N|                Y|          Five Stars|                  ok|2015-08-31 00:00:00|
|         US|   23526356|R273DCA6Y0H9V7|B00TCO0ZAA|     292641483|Professional 58mm...| 

In [ ]:
from pyspark.sql.functions import to_date
# Read in the Review dataset as a DataFrame

In [ ]:
# Create the customers_table DataFrame
customers_df = df.groupby("customer_id").agg({"customer_id":"count"}).withColumnRenamed("count(customer_id)", "customer_count")
customers_df.show()

+-----------+--------------+
|customer_id|customer_count|
+-----------+--------------+
|   52695798|             1|
|   48363612|             5|
|   46909180|             1|
|   45595220|             1|
|   50372387|             2|
|    9731896|             1|
|   24540309|             1|
|    2019000|             1|
|   50798385|             5|
|   37669883|             1|
|   19718301|             1|
|   45616772|             1|
|    2167730|             1|
|   47027968|             7|
|    5459822|             1|
|   52484883|             1|
|   12425248|             1|
|   15460750|             1|
|   46944960|             1|
|   37502310|             2|
+-----------+--------------+
only showing top 20 rows



In [20]:
# Create the products_table DataFrame and drop duplicates. 
products_df = df.select(["product_id", "product_title"]).drop_duplicates(["product_id"])
products_df.show()


+----------+--------------------+
|product_id|       product_title|
+----------+--------------------+
|0011300000|Genuine Geovision...|
|0974096512|DVD: Digital Phot...|
|0984445145|Tamara Lackey Cap...|
|0984445161|"Tamara Lackey's ...|
|0984920242|Samys Camera The ...|
|1123000034|NB-1LH NB-1L BATT...|
|1210300001|Orange Sources 12...|
|1379178304|Dengpin MC-DC2 Re...|
|1921453575|Pebble 8+2 Iridol...|
|198347598X|7 Colors Filters ...|
|3100028120|3 Years Warranty ...|
|3490204816|Orange Sources 3M...|
|5135000011|NP-BG1 NP-FG1 Dig...|
|6000005822|NB-2L Charger For...|
|6000006179|NB-6L Charger For...|
|6000006853|Best NP-40 NP-40N...|
|6000007388|KLIC-7004 Battery...|
|6000008775|NP-BK1 NPBK1 Type...|
|6000011474|EN-EL12 1500mAh B...|
|6000013159|2X EN-EL5 ENEL5 C...|
+----------+--------------------+
only showing top 20 rows



In [17]:
# Create the review_id_table DataFrame. 
# Convert the 'review_date' column to a date datatype with to_date("review_date", 'yyyy-MM-dd').alias("review_date")
review_id_df = df.select(["review_id", "customer_id", "product_id", "product_parent", to_date("review_date", 'yyyy-MM-dd').alias("review_date")])
review_id_df.show()

+--------------+-----------+----------+--------------+-----------+
|     review_id|customer_id|product_id|product_parent|review_date|
+--------------+-----------+----------+--------------+-----------+
|R1NBG94582SJE2|    2975964|B00I01JQJM|     860486164| 2015-08-31|
|R273DCA6Y0H9V7|   23526356|B00TCO0ZAA|     292641483| 2015-08-31|
| RQVOXO7WUOFK6|   52764145|B00B7733E0|      75825744| 2015-08-31|
|R1KWKSF21PO6HO|   47348933|B006ZN4U34|     789352955| 2015-08-31|
|R38H3UO1J190GI|   33680700|B00HUEBGMU|      19067902| 2015-08-31|
|R3NPIFKLR19NQA|   30301059|B008MW6Y12|     597683407| 2015-08-31|
|R3MBE6UCH3435E|   28282645|B00TE8XKIS|      35563334| 2015-08-31|
|R2E7A4FF0PVY5Q|     502818|B00ZKDUFBQ|     555190742| 2015-08-31|
| R3R8JDQ2BF4NM|    1481233|B010BZ7S2Q|     129544315| 2015-08-31|
|R1YND4BS823GN5|   27885926|B00HRXSSRA|     708418657| 2015-08-31|
|R2TZNSA18V7YF6|    3183883|B005C95NM4|     246957815| 2015-08-31|
|R22ZVRDPPXIDNL|   23208852|B00LBIBYBE|     746593019| 2015-08

In [ ]:
# Create the vine_table. DataFrame
vine_df = df.select(["review_id","star_rating","helpful_votes","total_votes","vine","verified_purchase"])
vine_df.show()

+--------------+-----------+-------------+-----------+----+-----------------+
|     review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|
+--------------+-----------+-------------+-----------+----+-----------------+
|R1NBG94582SJE2|          5|            0|          0|   N|                Y|
|R273DCA6Y0H9V7|          5|            0|          0|   N|                Y|
| RQVOXO7WUOFK6|          2|            1|          1|   N|                Y|
|R1KWKSF21PO6HO|          5|            0|          0|   N|                Y|
|R38H3UO1J190GI|          5|            1|          1|   N|                Y|
|R3NPIFKLR19NQA|          3|            0|          0|   N|                Y|
|R3MBE6UCH3435E|          3|            8|          8|   N|                N|
|R2E7A4FF0PVY5Q|          5|            0|          1|   N|                Y|
| R3R8JDQ2BF4NM|          5|            0|          2|   N|                Y|
|R1YND4BS823GN5|          1|            0|          0|   N|     

In [ ]:
# Store environmental variable
from getpass import getpass
password = getpass('Enter database password')
# Configure settings for RDS
mode = "append"
jdbc_url="jdbc:postgresql://database3.cfezeasis1an.us-east-2.rds.amazonaws.com:5432/BigData_Challenge"
config = {"user":"my_class",
          "password": password,
          "driver":"org.postgresql.Driver"}

Enter database password··········


In [21]:
# Write review_id_df to table in RDS
review_id_df.write.jdbc(url=jdbc_url, table='review_id_table', mode=mode, properties=config)

Py4JJavaError: ignored

In [ ]:
# Write products_df to table in RDS
# about 3 min
products_df.write.jdbc(url=jdbc_url, table='products_table', mode=mode, properties=config)

In [ ]:
# Write customers_df to table in RDS
# 5 min 14 s
customers_df.write.jdbc(url=jdbc_url, table='customers_table', mode=mode, properties=config)

In [ ]:
# Write vine_df to table in RDS
# 11 minutes
vine_df.write.jdbc(url=jdbc_url, table='vine_table', mode=mode, properties=config)